In [1]:
# ==============================================================
# Exact vs Approximate Model Counting - Test Harness
#  - include: utilities, exact brute force, Monte Carlo approx
#  - run on a few small formulas where exact is feasible
# ==============================================================

import math, random
from typing import Dict, Tuple
from sympy import symbols
from sympy.logic.boolalg import Boolean, And, Or, Not, Implies, Equivalent

# ------------------ Utilities ------------------

def vars_set(F: Boolean):
    return list(sorted(F.free_symbols, key=lambda s: s.name))

def eval_under(F: Boolean, assign: Dict):
    return bool(F.subs(assign))

def random_assignment(V, rng=random):
    return {v: bool(rng.getrandbits(1)) for v in V}

# ------------------ Exact (brute force) ------------------

def exact_model_count(F: Boolean, limit_vars: int = 22) -> Tuple[int, int]:
    V = vars_set(F)
    n = len(V)
    if n > limit_vars:
        raise ValueError(f"Exact count disabilitato per n={n} > {limit_vars}.")
    count = 0
    for mask in range(1 << n):
        assign = {V[i]: bool((mask >> i) & 1) for i in range(n)}
        if eval_under(F, assign):
            count += 1
    return count, n

# ------------------ Approximate (Monte Carlo) ------------------

def required_samples_eps_delta(eps: float, delta: float) -> int:
    if not (0 < eps < 1):   raise ValueError("eps deve essere in (0,1).")
    if not (0 < delta < 1): raise ValueError("delta deve essere in (0,1).")
    return math.ceil((1.0 / (2.0 * eps * eps)) * math.log(2.0 / delta))

def inverse_normal_cdf(p: float) -> float:
    # Acklam approximation
    a = [ -3.969683028665376e+01,  2.209460984245205e+02,
          -2.759285104469687e+02,  1.383577518672690e+02,
          -3.066479806614716e+01,  2.506628277459239e+00 ]
    b = [ -5.447609879822406e+01,  1.615858368580409e+02,
          -1.556989798598866e+02,  6.680131188771972e+01,
          -1.328068155288572e+01 ]
    c = [ -7.784894002430293e-03, -3.223964580411365e-01,
          -2.400758277161838e+00, -2.549732539343734e+00,
           4.374664141464968e+00,  2.938163982698783e+00 ]
    d = [ 7.784695709041462e-03,  3.224671290700398e-01,
          2.445134137142996e+00,  3.754408661907416e+00 ]
    plow, phigh = 0.02425, 1 - 0.02425
    if p < plow:
        q = math.sqrt(-2*math.log(p))
        x = (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5]) / \
            ((((d[0]*q + d[1])*q + d[2])*q + d[3])*q + 1)
    elif p > phigh:
        q = math.sqrt(-2*math.log(1 - p))
        x = -(((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5]) / \
              ((((d[0]*q + d[1])*q + d[2])*q + d[3])*q + 1)
    else:
        q = p - 0.5; r = q*q
        x = (((((a[0]*r + a[1])*r + a[2])*r + a[3])*r + a[4])*r + a[5])*q / \
            (((((b[0]*r + b[1])*r + b[2])*r + b[3])*r + b[4])*r + 1)
    e = 0.5 * (1 + math.erf(x / math.sqrt(2))) - p
    u = e * math.sqrt(2*math.pi) * math.exp(x*x/2)
    return x - u / (1 + x*u/2)

def clopper_pearson_ci_simple(k: int, N: int, alpha: float) -> Tuple[float, float]:
    if N == 0: return (0.0, 1.0)
    p_hat = k / N
    z = inverse_normal_cdf(1 - alpha/2)
    se = math.sqrt(p_hat * (1 - p_hat) / N) if 0 < p_hat < 1 else math.sqrt(0.25 / N)
    lo = max(0.0, p_hat - z * se)
    hi = min(1.0, p_hat + z * se)
    return (lo, hi)

def approx_model_count(F: Boolean, eps: float = 0.05, delta: float = 0.05,
                       max_samples: int = None, rng: random.Random = None) -> dict:
    rng = rng or random.Random()
    V = vars_set(F); n = len(V)
    N = required_samples_eps_delta(eps, delta)
    if max_samples is not None: N = min(N, max_samples)
    k = 0
    for _ in range(N):
        A = random_assignment(V, rng)
        if eval_under(F, A): k += 1
    p_hat = k / N
    lo_p, hi_p = clopper_pearson_ci_simple(k, N, alpha=delta)
    factor = 2**n
    return {
        "n": n, "N": N, "k": k,
        "p_hat": p_hat, "p_ci": (lo_p, hi_p),
        "count_hat": p_hat * factor,
        "count_ci": (lo_p * factor, hi_p * factor),
        "eps": eps, "delta": delta,
    }

# ------------------ Test Runner ------------------

def compare_exact_vs_approx(name: str, F: Boolean,
                            eps=0.05, delta=0.05, seed=42, max_samples=None):
    print(f"\n=== {name} ===")
    print(f"Formula: {F}")
    exact, n = exact_model_count(F)
    print(f"Exact  : #SAT = {exact}  over 2^{n} = {2**n}")

    res = approx_model_count(F, eps=eps, delta=delta,
                             max_samples=max_samples, rng=random.Random(seed))
    est = res["count_hat"]; lo, hi = res["count_ci"]
    rel_err = abs(est - exact) / max(1, exact)
    print(f"Approx : #SAT ≈ {est:.3f}  (CI {int((1-delta)*100)}%: [{lo:.3f}, {hi:.3f}])")
    print(f"samples N = {res['N']}, successes k = {res['k']}, p_hat = {res['p_hat']:.6f}")
    print(f"Relative error (point) = {100*rel_err:.2f}%")

if __name__ == "__main__":
    # Vars
    A, B, C, D, E = symbols('A B C D E')

    # 1) Facile: AND su 2 variabili -> 1 modello su 4
    F1 = And(A, B)  # #SAT = 1, n=2
    # 2) Classico "7 su 8": falso solo per A=B=C=True
    F2 = Or(Not(A), Not(B), Not(C))  # #SAT = 7, n=3
    # 3) Formula a 5 variabili, non banale
    F3 = And(
        Or(A, Not(B)),
        Or(B, C),
        Implies(C, D),
        Or(Not(D), E),
        Equivalent(A, Not(E))
    )

    # Confronti (eps=5%, delta=5%). Puoi ridurre eps per più precisione (aumenta N)
    compare_exact_vs_approx("F1: And(A,B)", F1, eps=0.03, delta=0.05, seed=7)
    compare_exact_vs_approx("F2: 7/8 clause", F2, eps=0.03, delta=0.05, seed=7)
    compare_exact_vs_approx("F3: 5-var mixed", F3, eps=0.03, delta=0.05, seed=7)


=== F1: And(A,B) ===
Formula: A & B
Exact  : #SAT = 1  over 2^2 = 4
Approx : #SAT ≈ 0.952  (CI 95%: [0.878, 1.026])
samples N = 2050, successes k = 488, p_hat = 0.238049
Relative error (point) = 4.78%

=== F2: 7/8 clause ===
Formula: ~A | ~B | ~C
Exact  : #SAT = 7  over 2^3 = 8
Approx : #SAT ≈ 7.013  (CI 95%: [6.899, 7.127])
samples N = 2050, successes k = 1797, p_hat = 0.876585
Relative error (point) = 0.18%

=== F3: 5-var mixed ===
Formula: (Implies(C, D)) & (B | C) & (Equivalent(A, ~E)) & (A | ~B) & (E | ~D)
Exact  : #SAT = 2  over 2^5 = 32
Approx : #SAT ≈ 2.248  (CI 95%: [1.894, 2.602])
samples N = 2050, successes k = 144, p_hat = 0.070244
Relative error (point) = 12.39%
